## Alumno: Gerardo de Miguel González

### A. Funciones

1. Escribe una función de R que reciba un vector de números x (la muestra), y genere una muestra bootstrap de ese vector.

In [17]:
set.seed(1)

In [18]:
#::GMG::Mi función con replicate() y sample()
#::nota::implementación ineficiente
#mi.bootstrap <- function (v) {
#    return (replicate(n = length(v), expr = sample(x = v, size = 1, replace = TRUE)))
#}
# https://www.rdocumentation.org/packages/base/versions/3.5.2/topics/sample
mi.bootstrap <- function (v) {
    return(sample(v, replace = TRUE))
}

In [19]:
#::GMG::Ejemplo
x <- c(5,3,1,7,9,2)
mi.bootstrap(x)

[1] 3 1 7 2 3 2

2. Utilizando la función anterior, escribe una función que reciba un vector de números x (la muestra original) y un número natural N, y que genere una matrix que tenga N columnas, siendo cada una una de las muestras de bootstrap.

In [20]:
#::GMG::Mi función, de nuevo con replicate(), ahora genero una matriz de length(v)xN
genera.bootstrap <- function(v, N = 10000) {
    return(replicate(n = N,expr = mi.bootstrap(v)))
}

In [21]:
#::GMG::Ejemplo
matriz.bootstraps <- genera.bootstrap(x,10)
dim(matriz.bootstraps)
print(matriz.bootstraps)

[1]  6 10

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]    2    9    1    3    1    9    9    9    5     2
[2,]    7    1    9    1    7    5    7    9    5     3
[3,]    7    9    2    5    1    9    7    1    3     1
[4,]    5    1    3    1    3    1    9    2    7     3
[5,]    3    9    7    2    9    9    5    1    7     7
[6,]    3    2    5    1    9    7    1    3    1     3


3. Escribe una función de R que reciba un vector de números x (la muestra) y genere una matriz que contenta TODAS las muestras jacknife con el mismo formato del ejercicio anterior.

In [22]:
#::GMG::Implemento mi función que genera jackknife "N-1"
genera.jackknife <- function(v) {
    M <- matrix(nrow = length(v)-1, ncol = length(v))
    for (i in 1:length(v)) {
        M[,i] <- v[-i]
    }
    return(M)
}

In [24]:
#::GMG::Ejemplo
matriz.jackknife <-genera.jackknife(x)
dim(matriz.jackknife)
print(matriz.jackknife)

[1] 5 6

     [,1] [,2] [,3] [,4] [,5] [,6]
[1,]    3    5    5    5    5    5
[2,]    1    1    3    3    3    3
[3,]    7    7    7    1    1    1
[4,]    9    9    9    9    7    7
[5,]    2    2    2    2    2    9


### B. Bootstrap

4. Utilizando las funciones anteriores considera la desviación estándar de la media muestral para una muestra de N=10000 que se distribuya como en el ejercicio 1 (gaussiana centrada en 1.78m y sigma=0.1m). Compara la desviación estándar obtenida, con la obtenida con bootstrap.

**::GMG::** Aquí conocemos la variable aleatoria X, su pdf, $X \sim \mathcal{N}\left(\mu_X, {\sigma_X}\right)$, su media poblacional $\mu_X$ y su desviación tipica poblacional $\sigma_X$ (y por lo tanto su varianza poblacional $\sigma_X^2$)

In [25]:
#::GMG::Genero la muestra de tamaño 10000 con distribución normal que me piden
set.seed(1)
mu <- 1.78
sigma <- 0.1
muestra <- rnorm(n = 10000,mean = 1.78,sd =0.1)

In [26]:
paste('Una muestra de tamaño',length(muestra),
      'con media muestral', round(mean(muestra),5),
      'y varianza muestral',round(var(muestra),5),
      'y desviación estándar muestral',round(sd(muestra),5))
paste('La media de la población es',mu,
      'la varianza de la población',sigma^2,
      'y finalmente, la desviación estándar',sigma)

[1] "Una muestra de tamaño 10000 con media muestral 1.77935 y varianza muestral 0.01025 y desviación estándar muestral 0.10124"

[1] "La media de la población es 1.78 la varianza de la población 0.01 y finalmente, la desviación estándar 0.1"

**::GMG::** Tenemos una muestra de tamaño n = 10000 sacada de la población con distribución $X \sim \mathcal{N}\left(\mu_X, {\sigma_X}\right)$. Hemos calculado tres estadísticos: la media muestral $\mu_{\bar x}$, la varianza muestral $\sigma_{\bar x}^2$ y la desviación estándar muestral $\sigma_{\bar x}$.

In [27]:
paste('La distribución muestral de las medias muestrales de TODAS las posibles muestras de tamaño', 
      length(muestra),
      'de la población, se aproximará a una distribución normal con media', 
      mu, 
      'varianza', 
      sigma^2/length(muestra),
      'y desviación estandar',
      sigma/sqrt(length(muestra))
     )

[1] "La distribución muestral de las medias muestrales de TODAS las posibles muestras de tamaño 10000 de la población, se aproximará a una distribución normal con media 1.78 varianza 1e-06 y desviación estandar 0.001"

**::GMG::** Es decir, que todas las posibles muestras de tamaño n de la población con distribución $X \sim \mathcal{N}\left(\mu_X, {\sigma_X}\right)$ van a tener un estadístico media muestral $\bar X$, también variable aleatoria, que se va a distribuir también de forma normal $\bar X \sim \mathcal{N}\left(\mu_{\bar x}, \sigma_{\bar x}\right)$ con $\sigma_{\bar x} = \frac {\sigma_X}{\sqrt n}$. Con el Bootstrap no vamos a tener las infinitas muestras de tamaño n pero si un gran número de muestras (1000).

In [28]:
#::GMG::Hacemos un Bootstrap con 1000 replicas (que no son TODAS las posibles, evidentemente)
set.seed(1)
matriz.replicas.bs <- genera.bootstrap(v = muestra, N = 1000)

In [29]:
dim(matriz.replicas.bs)

[1] 10000  1000

In [30]:
#::GMG::Calculamos los valores de las medias muestrales de cada réplica
medias.bootstrap <- apply(X = matriz.replicas.bs,MARGIN = 2,FUN = mean)
str(medias.bootstrap)

 num [1:1000] 1.78 1.78 1.78 1.78 1.78 ...


In [31]:
#::GMG::Calculamos la media de las medias de Bootstrap
media.bootstrap <- mean(medias.bootstrap)
paste('Estimación de la media del bootstrap:', round(media.bootstrap,5))

[1] "Estimación de la media del bootstrap: 1.77935"

In [32]:
#::GMG::Estimo la desviación estándar de los valores medios (medias) del bootstrap
sigma.bootstrap <- sd(medias.bootstrap)
paste('Estimación de la desviación estándar del bootstrap:', round(sigma.bootstrap,5))

[1] "Estimación de la desviación estándar del bootstrap: 0.00105"

**::GMG::** La desviación estándar obtenida en una sola muestra de tamaño 10000 es $\sigma_{\bar x}$ `0.10124`  y en el bootstrap $\sigma_B$ `0.00105` del orden de *100 veces menor* que es la $\sqrt{10000}$. Es lo que esperábamos. 

### C. Jackknife

5. Repite el ejercicio anterior utilizando la técnica jackknife. ¿Cuál da mejor resultado?   

In [33]:
#::GMG::repito el proceso con Jackknife
matriz.replicas.jk <- genera.jackknife(v = muestra)
medias.jk <- apply(X = matriz.replicas.jk,MARGIN = 2,FUN = mean)
str(matriz.replicas.jk)
str(medias.jk)

 num [1:9999, 1:10000] 1.8 1.7 1.94 1.81 1.7 ...
 num [1:10000] 1.78 1.78 1.78 1.78 1.78 ...


In [34]:
#::GMG::En el cálculo de la sd del jackknife hay que hacer una "corrección"
# https://en.wikipedia.org/wiki/Jackknife_resampling#Estimation
media.jk <- mean(medias.jk)
paste('Estimación de la media muestral del jackknife:', round(media.jk,5))
sigma.jk <- sqrt((length(medias.jk) - 1)*mean((medias.jk - media.jk)^2))
paste('Estimación de la desviación estándar del jackknife',round(sigma.jk,5))

[1] "Estimación de la media muestral del jackknife: 1.77935"

[1] "Estimación de la desviación estándar del jackknife 0.00101"

### D. Comparación Bootstrap vs Jackknife

**::GMG::** ¿Cuál es el mejor? **::PENDIENTE::**

**::DUDA**:: Para eso necesitamos conocer métricas de los estimadores *puntuales* que hemos obtenido con las técnicas de *resampling* que hemos usado.

1. ¿Cuál es la varianza y el sesgo de los etimadores?
2. ¿Cuál la *eficiencia relativa* de los estimadores?